In [1]:
from sklearn import tree
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import graphviz 
from sklearn.feature_extraction.text import CountVectorizer
from IPython.display import display
from pysupwsdpocket import PySupWSDPocket

# load data

In [2]:
df = pd.read_csv("./twitter_data/exploration_dataset.csv")

# load models and stopwords
For getting pysupwsdpocket to work it is necessary to [Download](https://supwsd.net/supwsd/downloads.jsp) the SupWSD Pocket model for english to the path: ```/home/your_user/pysupwsdpocket_models```

In [3]:
nlp = PySupWSDPocket(lang='en', model='semcor_omsti')
vectorizer = CountVectorizer()
stop_words = set(stopwords.words('english')) 

# Data preparation

In [18]:
corpus = []
Y = np.zeros(len(df['id']), dtype=int)
tweet_ids = []
for idx, row in df.iterrows():
    if row['tweet_id'] not in tweet_ids:
        print(row['text'])
        doc = nlp.wsd(row['text']) # word sense disambiguation
        sentence_senses = []
        for token in doc.tokens():
            if token.lemma not in stop_words:
                if token.max_probability()['id'] != 'U':
                    sentence_senses.append(token.max_probability()['synset'].name())
        corpus.append(" ".join(sentence_senses))
        tweet_ids.append(row['tweet_id'])
        if row['label'] != 'none':
            Y.append(1)
        else:
            Y.append(0)
corpus[:2]


These girls are the equivalent of the irritating Asian girls a couple years ago. Well done, 7. #MKR
Drasko they didn't cook half a bird you idiot #mkr
Hopefully someone cooks Drasko in the next ep of #MKR
of course you were born in serbia...you're as fucked as A Serbian Film #MKR
#MKR  Lost the plot - where's the big Texan with the elephant sized steaks that they all have for brekkie ?
So Drasko just said he was impressed the girls cooked half a chicken.. They cooked a whole one  #MKR
I've had better looking shits than these two! #MKR2015 #MKR #killerblondes
The face of very ugly promo girls ! Faces like cats arsehole #mkr excited to see them@go down tonight...literally http://t.co/HgoJrfoIeO
@mykitchenrules Elegant and beautiful?Cheap and trashy!Nothing more unattractive than girls banging on about how hot hey are. #mkr #notsassy
"He can't be a server at our restaurant, that beard makes him look like a terrorist." Everyone laughs. #fuckthanksgiving
Stop saying dumb blondes with pretty

AttributeError: 'JSONDecodeError' object has no attribute 'tokens'

In [19]:
X = vectorizer.fit_transform(corpus)
print(X)

(0, 86)	2
  (0, 0)	6
  (0, 69)	1
  (0, 51)	1
  (0, 1)	1
  (0, 206)	1
  (0, 15)	1
  (0, 199)	1
  (1, 0)	4
  (1, 48)	1
  (1, 137)	1
  (1, 90)	1
  (1, 29)	1
  (1, 98)	1
  (2, 0)	2
  (2, 1)	1
  (2, 48)	1
  (2, 140)	1
  (2, 80)	1
  (3, 0)	3
  (3, 130)	1
  (3, 87)	1
  (3, 126)	1
  (4, 0)	6
  (4, 115)	1
  :	:
  (54, 3)	1
  (54, 54)	1
  (54, 134)	1
  (54, 14)	1
  (54, 5)	1
  (55, 0)	7
  (55, 188)	1
  (55, 153)	1
  (55, 138)	1
  (55, 168)	1
  (55, 103)	1
  (55, 186)	1
  (55, 95)	1
  (56, 0)	4
  (56, 32)	1
  (56, 104)	1
  (56, 193)	1
  (56, 174)	1
  (57, 86)	1
  (57, 0)	6
  (57, 167)	1
  (57, 143)	1
  (57, 182)	1
  (57, 205)	1
  (57, 49)	1


In [20]:
corpus

['girl.n.01 equivalent.n.01 girl.n.01 couple.n.02 year.n.01 ago.r.01 well.r.01',
 'cook.n.01 one-half.n.01 bird.n.01 idiot.n.01',
 'person.n.01 cook.n.01 following.s.02',
 'naturally.r.01 give_birth.v.01 movie.n.01',
 'lose.v.01 plot.n.01 large.a.01 texan.n.01 elephant.n.01 steak.n.01',
 'state.v.01 impress.v.02 girl.n.01 cook.v.01 one-half.n.01 cook.v.02 whole.a.01 one.n.01',
 'well.r.02 look.v.02 bullshit.n.01 two.n.01',
 'face.n.03 ugly.a.01 girl.n.01 face.n.05 cat.n.01 excite.v.01 see.v.01 tonight.r.01',
 'elegant.a.01 nothing.n.01 unattractive.s.02 female_child.n.01 slam.v.02 hot.a.01',
 'restaurant.n.01 beard.n.01 make.v.02 look_like.v.01 laugh.v.01',
 'discontinue.v.01 allege.v.01 dense.s.04 blond.n.01 pretty.s.01 face.n.05 want.v.02 pretty.s.01 face.n.05 pull.v.01',
 'restaurant.n.01 look_like.v.01 small.a.01 female_child.n.01 room.n.01',
 'like.v.02 hot.a.01 pot.n.01 particularly.r.01 intoxicated.a.01 unconscious.a.01',
 'thaw.n.01 put.v.01 serve.v.07 dense.s.04',
 'girl.n.01 